In [1]:
import pandas as pd

In [2]:
games = pd.read_csv("./clean_datasets/games.csv")  # gameid, title, genres, description
purchased_games = pd.read_csv("./clean_datasets/purchased_games.csv")  # Dữ liệu game đã mua


In [3]:
# Nếu library là danh sách game (chuỗi), cần chuyển đổi thành danh sách thực
purchased_games['library'] = purchased_games['library'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Chuyển đổi các phần tử trong danh sách 'library' thành chuỗi
purchased_games['library'] = purchased_games['library'].apply(lambda x: [str(i) for i in x] if isinstance(x, list) else [])

# Đếm số lượng người chơi đã mua từng game
game_popularity = purchased_games.explode('library').groupby('library').size().reset_index(name='purchase_count')

# Đổi tên cột để khớp với gameid
game_popularity.rename(columns={'library': 'gameid'}, inplace=True)

# Chuyển đổi lại kiểu dữ liệu của gameid thành chuỗi
game_popularity['gameid'] = game_popularity['gameid'].astype(str)
games['gameid'] = games['gameid'].astype(str)

# Gộp với bảng games
games_with_popularity = games.merge(game_popularity, on="gameid", how="left")

# Điền giá trị 0 cho những game không có lượt mua nào
games_with_popularity['purchase_count'] = games_with_popularity['purchase_count'].fillna(0)


In [4]:
# Sắp xếp theo số lượng mua giảm dần
games_sorted = games_with_popularity.sort_values(by="purchase_count", ascending=False)

# Lấy 10% game phổ biến nhất
top_10_percent_games = games_sorted.head(int(len(games) * 0.1))

In [5]:
top_10_percent_games

,gameid,title,developers,publishers,genres,supported_languages,release_date,purchase_count
61496,730,Counter-Strike 2,['Valve'],['Valve'],"['Action', 'Free To Play']","['Czech', 'Danish', 'Dutch', 'English', 'Finni...",2012-08-21,43967.0
59961,578080,PUBG: BATTLEGROUNDS,['PUBG Corporation'],"['KRAFTON, Inc.']","['Action', 'Adventure', 'Massively Multiplayer...","['English', 'Korean', 'Simplified Chinese', 'F...",2017-12-21,28356.0
89627,550,Left 4 Dead 2,['Valve'],['Valve'],['Action'],"['Danish', 'Dutch', 'English', 'Finnish', 'Fre...",2009-11-16,24727.0
61671,218620,PAYDAY 2,['OVERKILL - a Starbreeze Studio.'],['Starbreeze Publishing AB'],"['Action', 'RPG']","['English', 'German', 'French', 'Italian', 'Sp...",2013-08-13,23785.0
84616,304930,Unturned,['Smartly Dressed Games'],['Smartly Dressed Games'],"['Action', 'Adventure', 'Casual', 'Indie', 'Fr...",['English'],2017-07-07,21620.0
...,...,...,...,...,...,...,...,...
86298,440550,Atomik: RunGunJumpGun,['ThirtyThree'],['Good Shepherd Entertainment'],"['Action', 'Indie']","['English', 'French', 'Italian', 'German', 'Sp...",2016-08-31,211.0
41976,501950,Rogue Islands,['Big Fat Alien'],Unknown,"['Action', 'Adventure', 'Indie', 'RPG', 'Strat...",['English'],2017-09-12,211.0
43075,462530,8i - Make VR Human,['8i'],['8i'],['Adventure'],['English'],2016-04-25,211.0
3464,1329540,CROSSBOW: Bloodnight,['Hyperstrange'],['Hyperstrange'],['Action'],"['English', 'Polish']",2020-09-24,211.0


In [ ]:
import os
if not os.path.exists("./data_model"):
    os.makedirs("./data_model")

top_10_percent_games.to_csv("./data_model/top_10_percent_games.csv", index=False)